In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
#Load Dataset
df= pd.read_csv('C:/Users/RAJAT/Desktop/Projects/mushrooms.csv')

In [4]:
df.head(n=5)
# the data is in alphabetical form. we will convert the data into numerical form
df.shape
# 23 features with each mushrooms
df.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [5]:
#data is in categorial form. we need to convert the data in numerical
le=LabelEncoder()
#by this we can define a function on pandas dataframe
#this function will convert our dataframe to numerical dataframe
ds= df.apply(le.fit_transform)
ds.head(n=5)

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [6]:
#converting it into a numpy array so that many operations can take place
data=ds.values
type(data)
#first colum is the label, rest are all features
data_y=data[:,0]
data_x=data[:,1:]
print(data_y)


[1 0 0 ... 0 1 0]


In [7]:
#splitting of data by itself
x_train,x_test,y_train,y_test= train_test_split(data_x,data_y,test_size=0.2)


In [8]:
# probability function
# we define the label by ourselves
def prior_prob(y_train,label):
    
    total_examples = y_train.shape[0]
    class_examples = np.sum(y_train==label) # number of y which is equal to label 
    
    return (class_examples)/float(total_examples) # ratio: no. of y/ total no. of y


In [9]:
# by defining label , it helps us calculate for one particular label. there are 2 types of label. we are only considering the one and ecluding the rers

y = np.array([0,5,5,1,1,1,1,0,0,0])
#np.sum(y==1)

#prior_prob(y,1)
#either we compute for all the labels individually or we make use of the function np.array and use only thos which we define as labels
# rest everything will be treated as false. only the label which is considered will be trested as true

In [10]:
# conditional probabilty
#filtering out the features by providing the label
#x_train has all the columns
#y_train will help us define the label 
#feature_col, we will compute probability of the particular column with label specified from y_tarin
def cond_prob(x_train,y_train,feature_col,feature_val,label):
    
    x_filtered = x_train[y_train==label] #filter x_train features  because that one particular column has mutile features with label(column) as specified
    numerator = np.sum(x_filtered[:,feature_col]==feature_val) # from that column pick features which are of feature_val(thisis multiple) type 
    denominator = np.sum(y_train==label)  #total number of labels
    
    return numerator/float(denominator)


In [19]:
### compution poterior probability

def predict(x_train,y_train,xtest):
    """Xtest is a single testing point, n features"""
    
    classes = np.unique(y_train) #type of code y has
    n_features = x_train.shape[1]
    post_probs = [] # List of prob for all classes and given a single testing point
    #Compute Posterior for each class 0 and 1 #label
    for label in classes:
        
        #Post_c = likelihood*prior
        likelihood = 1.0 #conditional probability
        for f in range(n_features): # conditional probability for each feature wrt label
            cond = cond_prob(x_train,y_train,f,xtest[f],label)
            likelihood *= cond  #product of all the probabilities
            
        prior = prior_prob(y_train,label)
        post = likelihood*prior
        post_probs.append(post) #append
        
    pred = np.argmax(post_probs) # saw maximum propability from that list
    return pred

In [20]:
output= predict(x_train,y_train,x_test[1]) #x_test is a training point which has the array of points of 1 column
print(output)
print(y_test[1])

1
1


In [29]:
#accuracy
def score(x_train,y_train,x_test,y_test):
    pred=[]
    for i in range(x_test.shape[0]):
        pred_label= predict(x_train,y_train,x_test[i])
        pred.append(pred_label)
        
    pred=np.array(pred)
    accuracy=np.sum(pred==y_test)/y_test.shape[0]
    
    return accuracy

In [30]:
print(score(x_train,y_train,x_test,y_test))

0.9987692307692307
